In [ ]:
"""
Volcanic Activity Classifier with Voice Alarm (PyQt5 + Keras)

Author: Dr. Yasir Hussein Shakir
Note: If you face any issues with this code, please contact me.
"""

import sys
import numpy as np
import pyttsx3
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QPushButton, QVBoxLayout, QFileDialog, QFrame
)
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# ===============================
# Load Trained Model
# ===============================
model = load_model("DCNNVA_model.h5")

# Define your class names
class_names = ["No Active", "Active"]

# ===============================
# PyQt5 App
# ===============================
class VolcanicClassifier(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Volcanic Activity Classifier with Alarm")
        self.setGeometry(200, 100, 700, 650)

        # TTS Engine
        self.engine = pyttsx3.init()
        self.engine.setProperty("rate", 150)   # Speed of speech
        self.engine.setProperty("volume", 1.0) # Volume (0.0 to 1.0)

        # Apply Styles
        self.setStyleSheet("""
            QWidget {
                background-color: #121212;
                color: #f5f5f5;
                font-family: 'Segoe UI', Arial, sans-serif;
            }
            QLabel#titleLabel {
                font-size: 28px;
                font-weight: bold;
                color: #FFB300;
                margin: 20px;
            }
            QLabel#resultLabel {
                font-size: 20px;
                font-weight: bold;
                color: #4CAF50;
                margin: 15px;
                padding: 12px;
                border-radius: 10px;
                background-color: #1e1e1e;
            }
            QPushButton {
                background-color: #2196F3;
                color: white;
                border-radius: 8px;
                padding: 12px 20px;
                font-size: 16px;
                font-weight: bold;
                border: none;
            }
            QPushButton:hover {
                background-color: #42A5F5;
            }
            QFrame#imageFrame {
                background-color: #2d2d2d;
                border: 2px solid #444;
                border-radius: 12px;
                margin: 20px;
            }
        """)

        # Title
        self.label = QLabel(" Volcanic Activity Detection", self)
        self.label.setObjectName("titleLabel")
        self.label.setAlignment(Qt.AlignCenter)

        # Image frame
        self.image_frame = QFrame(self)
        self.image_frame.setObjectName("imageFrame")
        self.image_label = QLabel(self.image_frame)
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setFixedSize(400, 400)

        # Result Label
        self.result_label = QLabel("", self)
        self.result_label.setObjectName("resultLabel")
        self.result_label.setAlignment(Qt.AlignCenter)

        # Upload Button
        self.upload_btn = QPushButton("📂 Upload Image", self)
        self.upload_btn.clicked.connect(self.load_image)

        # Layout
        layout = QVBoxLayout()
        layout.addWidget(self.label)
        layout.addWidget(self.image_frame, alignment=Qt.AlignCenter)
        layout.addWidget(self.result_label, alignment=Qt.AlignCenter)
        layout.addWidget(self.upload_btn, alignment=Qt.AlignCenter)

        frame_layout = QVBoxLayout(self.image_frame)
        frame_layout.addWidget(self.image_label, alignment=Qt.AlignCenter)

        self.setLayout(layout)

    def speak(self, message):
        """Speak one-time message using pyttsx3."""
        self.engine.stop()  # stop any previous speech
        self.engine.say(message)
        self.engine.runAndWait()

    def load_image(self):
        file_path, _ = QFileDialog.getOpenFileName(
            self, "Select Image", "", "Image Files (*.png *.jpg *.jpeg)"
        )
        if file_path:
            # Display image
            pixmap = QPixmap(file_path).scaled(
                400, 400, Qt.KeepAspectRatio, Qt.SmoothTransformation
            )
            self.image_label.setPixmap(pixmap)

            # Preprocess image
            img = image.load_img(file_path, target_size=(224, 224))
            img_array = image.img_to_array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)

            # Prediction
            prediction = model.predict(img_array)
            predicted_class = class_names[np.argmax(prediction)]
            confidence = np.max(prediction) * 100

            # Update result label
            self.result_label.setText(
                f"<b>Prediction:</b> {predicted_class}<br>"
                f"<b>Confidence:</b> {confidence:.2f}%"
            )

            # Voice Alarm (one-time per detection)
            if predicted_class == "Active":
                self.speak("Warning! Volcanic Activity Detected.")
            else:
                self.speak("No volcanic activity detected.")

# ===============================
# Run the App
# ===============================
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = VolcanicClassifier()
    window.show()
    sys.exit(app.exec_())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
